In [1]:
!pip install ggplot
!pip install matplotlib
!pip install bokeh
!pip install dask
!pip install statsmodels
!pip install cloudpickle
!pip install requests
!pip install tpot
!pip install lightgbm
!pip install networkx

    100% |████████████████████████████████| 2.2MB 4.1MB/s 
    100% |████████████████████████████████| 14.7MB 2.3MB/s 
  Running setup.py bdist_wheel for bokeh ... done
  Stored in directory: /content/.cache/pip/wheels/ff/28/51/22e8d08e9d5383ee1de981aaa8ff7bc53c7d65022e5101400f
Successfully built bokeh
    100% |████████████████████████████████| 604kB 2.6MB/s 
    80% |█████████████████████████▉      | 716kB 3.0MB/s eta 0:00:01    100% |████████████████████████████████| 890kB 3.4MB/s 
    100% |████████████████████████████████| 942kB 5.2MB/s 
    100% |████████████████████████████████| 51kB 7.2MB/s 
  Running setup.py bdist_wheel for tpot ... done
  Stored in directory: /content/.cache/pip/wheels/00/f3/4d/8d28f69c59669ba720eebaa4ebc8a3b28da2a61662367b38e8
  Running setup.py bdist_wheel for deap ... done
  Stored in directory: /content/.cache/pip/wheels/22/ea/bf/dc7c8a2262025a0ab5da9ef02282c198be88902791ca0c6658
  Running setup.py bdist_wheel for stopit ... done
  Stored in directory: /

In [2]:
import seaborn as sns
from ggplot import *
from matplotlib import pyplot as plt
import bokeh

import pandas as pd
import dask.dataframe as dd
import numpy as np
import scipy as sc
import statsmodels as sm
import networkx as nx

import sklearn as sk
import tensorflow as tf
import keras
import xgboost as xgb
import lightgbm as lgbm
import tpot

import sys
import os
import gc

/usr/local/lib/python3.6/dist-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/usr/local/lib/python3.6/dist-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/usr/local/lib/python3.6/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


#Data sources

##DNA, Mutation
Literally, per genome and chromosome the change in the pair compared to a normal reference. Remember we have (Adenine,Thymine) and (Guanine,Cytosine) as the base pairs.




The types of mutations include (taken from here):

**Missense mutation**:. This type of mutation is a change in one DNA base pair that results in the substitution of one amino acid for another in the protein made by a gene.

**Nonsense mutation**: is also a change in one DNA base pair. Instead of substituting one amino acid for another, however, the altered DNA sequence prematurely signals the cell to stop building a protein. This type of mutation results in a shortened protein that may function improperly or not at all.

**Insertion**: An insertion changes the number of DNA bases in a gene by adding a piece of DNA. As a result, the protein made by the gene may not function properly.

**Deletion**: A deletion changes the number of DNA bases by removing a piece of DNA. Small deletions may remove one or a few base pairs within a gene, while larger deletions can remove an entire gene or several neighboring genes. The deleted DNA may alter the function of the resulting protein(s).

**Duplication**: A duplication consists of a piece of DNA that is abnormally copied one or more times. This type of mutation may alter the function of the resulting protein.

**Frameshift mutation**: This type of mutation occurs when the addition or loss of DNA bases changes a gene's reading frame. A reading frame consists of groups of 3 bases that each code for one amino acid. A frameshift mutation shifts the grouping of these bases and changes the code for amino acids. The resulting protein is usually nonfunctional. Insertions, deletions, and duplications can all be frameshift mutations.

**Repeat expansion**: Nucleotide repeats are short DNA sequences that are repeated a number of times in a row. For example, a trinucleotide repeat is made up of 3-base-pair sequences, and a tetranucleotide repeat is made up of 4-base-pair sequences. A repeat expansion is a mutation that increases the number of times that the short DNA sequence is repeated. This type of mutation can cause the resulting protein to function improperly.

### DATA FIELDS, shape (422553, 11)
```
ID      |  Location        | Change     |  Gene   | Mutation type|  Var.Allele.Frequency  | Amino acid

SampleID,| Chr, Start, Stop|  Ref, Alt  | Gene    |    Effect    |  DNA_VAF, RNA_VAF      | Amino_Acid_Change

string   |string, int, int | char, char | string  |    string    |  float, float          |  string
```

**NOTE**: this gives us direct insight in how genetic mutations lead to changes in amino-acids.

## Copy Number Variations
A copy number variation (CNV) is when the number of copies of a particular gene varies from one individual to the next.

### DATA FIELDS, shape (24802, 372)
```
Gene      | Chr, Start, Stop | Strand     |   SampleID 1..SampleID N

string    |string, int, int  | int        |  int..int
```

## Methylation, gene expression regulation
Degree of methylation indicates addition of Methyl groups to the DNA. Increased methylation is associated with less transcription of the DNA: Methylated means the gene is switched OFF, Unmethylated means the gene is switched ON.

Alterations of DNA methylation have been recognized as an important component of cancer development.

### DATA FIELDS, shape (485577, 483)
```
probeID   | Chr, Start, Stop | Strand  | Gene   |  Relation_CpG_island | SampleID 1..SampleID N

string    |string, int, int  | int     | string |   string             | float..float
```

## RNA, gene expression
Again four building blocks; Adenosine (A), Uracil (U), Guanine (G), Cytosine (C).

(DNA) --> (RNA)

A --> U

T --> A

C --> G

G --> C

Gene expression profiles, continuous values resulting from the normalisation of counts.

### DATA FIELDS, shape (60531, 477)
```
Gene      | Chr, Start, Stop | Strand  | SampleID 1..SampleID N

string    |string, int, int  | int     |  float..float
```

## miRNA, transcriptomics
The connection between the RNA production and protein creation. I.e. perhaps miRNA expression values can be associated with specific proteins.

### DATA FIELDS, shape (2220, 458)
```
MIMATID  | Name   | Chr, Start, Stop | Strand  | SampleID 1..SampleID N

string   | string |string, int, int  | int     |  float..float
```

## Proteomes
Proteine expression profiles, ditto, continuous values resulting from the normalisation of counts

### DATA FIELDS, shape (282, 355)
```
ProteinID  | SampleID 1..SampleID N

string     | float..float
```

**QUIZ**, identify our data sets in the following image!

![Quiz](https://media.nature.com/m685/nature-assets/nrg/journal/v16/n2/images/nrg3868-f1.jpg)

# GOAL
**Some degree of multi-omic or trans-omic analysis and identification of pathways.**


![Quiz](https://www.cell.com/cms/attachment/2119084140/2088971044/gr1_lrg.jpg)



## Our reality
![Quiz](https://media.springernature.com/m685/nature-assets/nrg/journal/v16/n2/images/nrg3868-f2.jpg)

In [3]:
Melanoma_CopyNumberVariations = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_CNV.txt", sep="\t")
Melanoma_Mutation = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_Mutation.txt", sep="\t")
Melanoma_Methylation = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_Methylation.txt", sep="\t")

Melanoma_GeneExpression = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_GeneExpression.txt", sep="\t")
Melanoma_miRNA = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_miRNA.txt", sep="\t")
Melanoma_Proteome = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_Proteome.txt", sep="\t")

Melanoma_Phenotype_Metadata = pd.read_table("https://storage.googleapis.com/genx_2018/Melanoma_Phenotype_Metadata.txt", sep="\t")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
